In [1]:
import numpy as np # linear algebra
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import sklearn
import sklearn.metrics

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from math import sqrt

In [21]:
#building all kinds of evaluating parameters
from sklearn.metrics import classification_report, accuracy_score  
from sklearn.metrics import precision_score, recall_score 
from sklearn.metrics import roc_curve,auc, roc_auc_score, average_precision_score
from sklearn.metrics import f1_score, matthews_corrcoef 
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import confusion_matrix 

In [4]:
from imblearn.pipeline import make_pipeline as make_pipeline_imb # To do our transformation in a unique time
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.model_selection import train_test_split
classifier = RandomForestClassifier

In [5]:
import pandas as pd

url2 = 'Dataset/PRFI_4M_DEF_HEAD.txt'
df = pd.read_table(url2, sep=';')

In [6]:
df["FLAG_FRODE"].replace({"T": 1, "F": 0}, inplace=True)

In [7]:
df["DT_OP_GMT"] = pd.to_datetime(df["DT_OP_GMT"])

start_datetr = '2016-03-01'
end_datetr = '2016-05-31'

mask_tr = (df['DT_OP_GMT'] >= start_datetr) & (df['DT_OP_GMT'] < end_datetr) 
df_train = df.loc[mask_tr]

print(df_train)

            ID_OPER  DT_OP_GMT ORA_OP_GMT COD_CANA  COD_CABC  COD_ESITO  \
0        6231261973 2016-03-14   11:27:28      ATM      6011          0   
1        6217512045 2016-03-09   09:48:09      INT      5942          0   
2        6204998318 2016-03-04   12:01:47      INT      5192          0   
3        6260986375 2016-03-25   08:29:03      INT      5968          0   
4        6214969493 2016-03-08   10:36:09      POS      5411          0   
...             ...        ...        ...      ...       ...        ...   
5438812  6443141702 2016-05-30   19:07:33      INT      7273          0   
5439277  6442518610 2016-05-30   15:57:05      INT      5045          0   
5439283  6442463929 2016-05-30   17:44:10      INT      8999          0   
5439406  6442771077 2016-05-30   16:54:51      ATM      6011          0   
5439830  6396976594 2016-05-13   17:07:03      ATM      6011          0   

         PROCESSING_COD  IMP_TRZ_RECON FL_AUTORIZZAZIONE COD_ABC_PAES  ...  \
0                3661

In [8]:
start_datevl = '2016-06-14'
end_datevl = '2016-06-22'

start_datets = '2016-06-22'
end_datets = '2016-06-30'

#greater than the start date and smaller than the end date
mask_vl = (df['DT_OP_GMT'] >= start_datevl) & (df['DT_OP_GMT'] < end_datevl)
df_valid = df.loc[mask_vl]

#greater than the start date and smaller than the end date
mask = (df['DT_OP_GMT'] >= start_datets) & (df['DT_OP_GMT'] < end_datets)
df_test = df.loc[mask]


In [9]:
df_train = df_train[['ID_OPER', 'DT_OP_GMT', 'ORA_OP_GMT', 'COD_CANA', 'COD_CABC',
                'PROCESSING_COD', 'IMP_TRZ_RECON', 'FL_AUTORIZZAZIONE',
                'COD_ABC_PAES', 'ID_TIPO_CARTA', 'TIPO_INPUT_MODE',
                'COD_CIRC', 'FLAG_FRODE', 'TRX_TYPE', 'Row_Num', 'TIME_ORA_OP','DEV_MEDIA_IMP_TC_1M','PERC_RISK_CIRC_TC_TO_1M',
               'DEV_MEDIA_IMP_CAN_1M','DEV_MEDIA_IMP_TO_1M','PERC_OPER_NON_AUTOR_1M','PERC_OPER_NON_AUTOR_2M','MEDIA_NON_AUT_SEC_4',
               'DEV_IMP_MEDIA_ULTIME_3', 'PERC_RISK_MCC_1M','FLAG_IMP_PREC_0','TREND_3_5','FLAG_CAMBIO_PAESE_AUT',
               'FLAG_CAMBIO_PAESE_NON_AUT','PERC_ESITO_NSF_G','PERC_ESITO_CV2_G',
               'PERC_ESITO_PIN_G','PERC_ESITO_SCUDO_G','PERC_ESITO_DT_SCAD_G','PERC_ESITO_ALTRO_G',
               'DEV_MEDIA_AUT_SEC_4']]

In [10]:
from sklearn.preprocessing import RobustScaler

rob_scaler = RobustScaler()

df_train['IMP_TRZ_RECON_NORM'] = rob_scaler.fit_transform(df_train['IMP_TRZ_RECON'].values.reshape(-1,1)) 

df_train.drop(['IMP_TRZ_RECON'], axis=1, inplace=True)

In [11]:
# Number of data points in the minority class
df_train = df_train.reset_index()
number_records_fraud = len(df_train[df_train.FLAG_FRODE == 1])
fraud_indices = np.array(df_train[df_train.FLAG_FRODE == 1].index)

In [12]:
# Picking the indices of the normal classes
normal_indices = df_train[df_train.FLAG_FRODE == 0].index

# Out of the indices we picked, randomly select "x" number (number_records_fraud)
random_normal_indices = np.random.choice(normal_indices, number_records_fraud, replace = False)
random_normal_indices = np.array(random_normal_indices)

# Appending the 2 indices
under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])

# Under sample dfset
df_train_undersample = df_train.iloc[under_sample_indices,:]

#X_undersample = under_sample_df.loc[:, under_sample_df.columns != 'FLAG_FRODE']
#y_undersample = under_sample_df.loc[:, under_sample_df.columns == 'FLAG_FRODE']
df_train_undersample

,index,ID_OPER,DT_OP_GMT,ORA_OP_GMT,COD_CANA,COD_CABC,PROCESSING_COD,FL_AUTORIZZAZIONE,COD_ABC_PAES,ID_TIPO_CARTA,...,FLAG_CAMBIO_PAESE_AUT,FLAG_CAMBIO_PAESE_NON_AUT,PERC_ESITO_NSF_G,PERC_ESITO_CV2_G,PERC_ESITO_PIN_G,PERC_ESITO_SCUDO_G,PERC_ESITO_DT_SCAD_G,PERC_ESITO_ALTRO_G,DEV_MEDIA_AUT_SEC_4,IMP_TRZ_RECON_NORM
36,38,6226002651,2016-03-12,09:16:37,INT,7399,360000,T,BMU,1009,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,-1.000000,-0.319642
38,40,6251120205,2016-03-21,16:55:41,INT,5968,0,T,GBR,1028,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.061449,-0.171264
60,71,6277642583,2016-03-31,19:43:54,INT,5499,6300,T,GBR,1028,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,2.039607,-0.308585
101,112,6197847455,2016-03-02,01:47:32,INT,7995,116300,T,MLT,1028,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2052.826531,0.786003
102,113,6197847677,2016-03-02,01:46:43,INT,7995,116300,T,MLT,1028,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1057.342105,0.786003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220993,1463305,6215878143,2016-03-08,15:39:37,ATM,6011,16100,T,ITA,1063,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,634.189716,0.233180
3843025,4631165,6438369121,2016-05-28,17:47:52,ATM,6011,16100,T,ITA,1063,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,-0.171098,-0.098513
3361889,4050776,6244354101,2016-03-18,23:32:20,I3D,7995,6300,T,GBR,1085,...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,3.599343,2.997291
1116990,1337722,6316630238,2016-04-14,23:04:44,INT,6211,0,T,ITA,1028,...,1,1,0.0,0.0,0.0,0.0,0.0,0.0,-0.350765,2.814860


In [13]:
# Normalizziamo la variabile importo

rob_scaler = RobustScaler()

df_valid['IMP_TRZ_RECON_NORM'] = rob_scaler.fit_transform(df_valid['IMP_TRZ_RECON'].values.reshape(-1,1)) 

df_valid.drop(['IMP_TRZ_RECON'], axis=1, inplace=True)

c:\users\fap91\pycharmprojects\tesitirocinio\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\fap91\pycharmprojects\tesitirocinio\venv\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
df_test

,ID_OPER,DT_OP_GMT,ORA_OP_GMT,COD_CANA,COD_CABC,COD_ESITO,PROCESSING_COD,IMP_TRZ_RECON,FL_AUTORIZZAZIONE,COD_ABC_PAES,...,PERC_ESITO_ALL_G,PERC_ESITO_ALL_M,DEV_MEDIA_AUT_SEC_4,DEV_MEDIA_AUT_TO_SEC_4,MEDIA_NON_AUT_SEC_4,MEDIA_NON_AUT_SEC_3,MEDIA_NON_AUT_SEC_2,TIME_NON_AUT_SEC_LAST,ANNO,MESE
52,6517267568,2016-06-25,21:33:08,INT,5735,0,6300,2.28,T,LUX,...,0.0,0.111111,-0.978184,1.403085,3403015.75,3.860554e+06,2874068.0,794691,2016,6
605,6520176282,2016-06-27,10:00:59,INT,5968,0,0,1.00,T,LUX,...,0.0,0.000000,2.224835,2.630098,3798059.75,4.920078e+06,7163729.5,561800,2016,6
1037,6526204408,2016-06-29,10:16:29,POS,5946,0,0,100.00,T,ITA,...,0.0,0.923077,-0.967212,-0.993849,138550.50,1.847340e+05,277101.0,36,2016,6
1117,6526179755,2016-06-29,10:10:29,POS,5411,0,0,14.70,T,ITA,...,0.0,0.000000,138.043493,1.122938,2993654.50,3.991535e+06,5987230.0,730,2016,6
1118,6513804107,2016-06-24,17:11:32,INT,5735,0,6300,0.99,T,LUX,...,0.0,0.000000,-0.846943,-0.654957,2993654.50,3.991535e+06,5987230.0,624958,2016,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5442896,6514848090,2016-06-25,07:53:22,ATM,6011,0,16100,20.00,T,ITA,...,0.0,0.000000,-0.827383,0.000000,0.00,0.000000e+00,0.0,491092,2016,6
5442897,6527172347,2016-06-29,15:54:46,POS,5511,0,0,11.50,T,ITA,...,0.0,0.000000,1.565676,0.000000,0.00,0.000000e+00,0.0,84380,2016,6
5442899,6518417168,2016-06-26,14:58:45,ATM,6011,0,16100,40.00,T,ITA,...,0.0,0.000000,1.572586,0.000000,0.00,0.000000e+00,0.0,111923,2016,6
5442900,6527197771,2016-06-29,16:01:02,ATM,6011,0,16100,40.00,T,ITA,...,0.0,0.000000,574.515957,0.000000,0.00,0.000000e+00,0.0,376,2016,6


In [15]:
# Normalizziamo la variabile importo

rob_scaler = RobustScaler()

df_test['IMP_TRZ_RECON_NORM'] = rob_scaler.fit_transform(df_test['IMP_TRZ_RECON'].values.reshape(-1,1)) 

df_test.drop(['IMP_TRZ_RECON'], axis=1, inplace=True)

c:\users\fap91\pycharmprojects\tesitirocinio\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\fap91\pycharmprojects\tesitirocinio\venv\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
#separating the X and the Y values
    
xTrain = df_train_undersample[['TIME_ORA_OP','ID_TIPO_CARTA','DEV_MEDIA_IMP_TC_1M', 'PERC_RISK_CIRC_TC_TO_1M',
       'DEV_MEDIA_IMP_CAN_1M', 'DEV_MEDIA_IMP_TO_1M', 'PERC_OPER_NON_AUTOR_1M',
       'PERC_OPER_NON_AUTOR_2M', 'MEDIA_NON_AUT_SEC_4',
       'DEV_IMP_MEDIA_ULTIME_3', 'PERC_RISK_MCC_1M', 'FLAG_IMP_PREC_0',
       'TREND_3_5','FLAG_CAMBIO_PAESE_NON_AUT',
       'PERC_ESITO_NSF_G', 'PERC_ESITO_CV2_G', 'PERC_ESITO_PIN_G',
       'PERC_ESITO_SCUDO_G', 'PERC_ESITO_DT_SCAD_G', 'PERC_ESITO_ALTRO_G',
       'DEV_MEDIA_AUT_SEC_4','IMP_TRZ_RECON_NORM']]

xValid = df_valid[['TIME_ORA_OP','ID_TIPO_CARTA','DEV_MEDIA_IMP_TC_1M', 'PERC_RISK_CIRC_TC_TO_1M',
       'DEV_MEDIA_IMP_CAN_1M', 'DEV_MEDIA_IMP_TO_1M', 'PERC_OPER_NON_AUTOR_1M',
       'PERC_OPER_NON_AUTOR_2M', 'MEDIA_NON_AUT_SEC_4',
       'DEV_IMP_MEDIA_ULTIME_3', 'PERC_RISK_MCC_1M', 'FLAG_IMP_PREC_0',
       'TREND_3_5','FLAG_CAMBIO_PAESE_NON_AUT',
       'PERC_ESITO_NSF_G', 'PERC_ESITO_CV2_G', 'PERC_ESITO_PIN_G',
       'PERC_ESITO_SCUDO_G', 'PERC_ESITO_DT_SCAD_G', 'PERC_ESITO_ALTRO_G',
       'DEV_MEDIA_AUT_SEC_4','IMP_TRZ_RECON_NORM']]

xTest = df_test[['TIME_ORA_OP','ID_TIPO_CARTA','DEV_MEDIA_IMP_TC_1M', 'PERC_RISK_CIRC_TC_TO_1M',
       'DEV_MEDIA_IMP_CAN_1M', 'DEV_MEDIA_IMP_TO_1M', 'PERC_OPER_NON_AUTOR_1M',
       'PERC_OPER_NON_AUTOR_2M', 'MEDIA_NON_AUT_SEC_4',
       'DEV_IMP_MEDIA_ULTIME_3', 'PERC_RISK_MCC_1M', 'FLAG_IMP_PREC_0',
       'TREND_3_5','FLAG_CAMBIO_PAESE_NON_AUT',
       'PERC_ESITO_NSF_G', 'PERC_ESITO_CV2_G', 'PERC_ESITO_PIN_G',
       'PERC_ESITO_SCUDO_G', 'PERC_ESITO_DT_SCAD_G', 'PERC_ESITO_ALTRO_G',
       'DEV_MEDIA_AUT_SEC_4','IMP_TRZ_RECON_NORM']]

yTrain = df_train_undersample["FLAG_FRODE"]

yValid = df_valid["FLAG_FRODE"]

yTest = df_test["FLAG_FRODE"]

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [6,8,10],
    'max_features': [2,3],
    'n_estimators': [100, 200, 300]
}
# Create a based model
rf = RandomForestClassifier(random_state=0)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(xTrain, yTrain)
grid_search.best_params_

In [ ]:
print (f'Train Accuracy - : {grid_search.score(xTrain,yTrain):.3f}')
print (f'Test Accuracy - : {grid_search.score(xValid,yValid):.3f}')

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [6,8,10,12],
    'max_features': [2,3,4],
    'n_estimators': [100, 200, 300]
}
# Create a based model
rf = RandomForestClassifier(random_state=0)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(xTrain, yTrain)
grid_search.best_params_

In [ ]:
print (f'Train Accuracy - : {grid_search.score(xTrain,yTrain):.3f}')
print (f'Test Accuracy - : {grid_search.score(xValid,yValid):.3f}')

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [10,12,14],
    'max_features': [2,3,4],
    'n_estimators': [100, 200, 300]
}
# Create a based model
rf = RandomForestClassifier(random_state=0)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(xTrain, yTrain)
grid_search.best_params_

In [17]:
#building the Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier 
#random forest model creation 
rfc = RandomForestClassifier(bootstrap = True, random_state=0,
                             max_features = 4, max_depth = 12, n_estimators = 200)
rfc.fit(xTrain,yTrain)
predictions = rfc.predict(xTest)
print(accuracy_score(yTest, predictions))
print(confusion_matrix(yTest, predictions))
print(classification_report(yTest, predictions))

0.7897332329521237
[[195395  52647]
 [   139   2862]]
              precision    recall  f1-score   support

           0       1.00      0.79      0.88    248042
           1       0.05      0.95      0.10      3001

    accuracy                           0.79    251043
   macro avg       0.53      0.87      0.49    251043
weighted avg       0.99      0.79      0.87    251043



In [28]:
rfc.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [18]:
#building the Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier 
#random forest model creation 
rfc = RandomForestClassifier(bootstrap = True, random_state=0,
                             max_features = 2, max_depth = 14, n_estimators = 200)

rfc.fit(xTrain,yTrain)
predictions = rfc.predict(xTest)
print(accuracy_score(yTest, predictions))
print(confusion_matrix(yTest, predictions))
print(classification_report(yTest, predictions))

0.7898607011547822
[[195425  52617]
 [   137   2864]]
              precision    recall  f1-score   support

           0       1.00      0.79      0.88    248042
           1       0.05      0.95      0.10      3001

    accuracy                           0.79    251043
   macro avg       0.53      0.87      0.49    251043
weighted avg       0.99      0.79      0.87    251043



In [27]:
import numpy as np
cm = confusion_matrix(yTest, predictions)

tn = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tp = cm[1][1]

recall = tp/(tp+fn)
specificity = tn/(tn+fp)

# sensitivity, hit rate, recall, or true positive rate
print('Accuracy',(tp+tn)/(tp+fp+fn+tn))

# sensitivity, hit rate, recall, or true positive rate
print('Recall',tp/(tp+fn))

# specificity or true negative rate
print('Specificity', tn/(tn+fp) )

#F1 Score
print("F1 Score: " ,tp/(tp+((fn+fp)/2)))

#Precision
print("Precision: " ,tp/(tp+fp) )

roc_auc = roc_auc_score(yTest, predictions) 
print("The ROC AUC Score is {}".format(roc_auc)) 

avg_prec = average_precision_score(yTest, predictions) 
print("The Average precision Score is {}".format(avg_prec)) 

#Gmean
print('G-Mean',sqrt(recall*specificity))

# precision or positive predictive value
ppv = tp/(tp+fp)
# negative predictive value
npv = tn/(tn+fn)
# fall out or false positive rate
fpr = fp/(fp+tn)
# false negative rate
fnr = fn/(tp+fn)
# false discovery rate
fdr = fp/(tp+fp)

# overall accuracy
acc = (tp+tn)/(tp+fp+fn+tn)

Accuracy 0.7898607011547822
Recall 0.9543485504831722
Specificity 0.787870602559244
F1 Score:  0.09794466673506377
Precision:  0.05162127575205926
The ROC AUC Score is 0.8711095765212081
The Average precision Score is 0.049810412930701625
G-Mean 0.8671235018846613


## thresholds

In [1]:
from sklearn.metrics import precision_recall_curve
# predict probabilities
rfc_probs = rfc.predict_proba(xTest)
rfc_probs

NameError: name 'rfc' is not defined

In [37]:
from sklearn.metrics import f1_score
from numpy import arange

# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

probs = rfc_probs[:,1]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [f1_score(yTest, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.719, F-Score=0.17233


In [39]:
from sklearn.metrics import f1_score
from numpy import arange

# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

probs = rfc_probs[:,1]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [roc_auc_score(yTest, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, AucScore=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.552, AucScore=0.87701


In [44]:
from sklearn.metrics import f1_score
from numpy import arange

# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

probs = rfc_probs[:,0]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [accuracy_score(yTest, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, accuracy_score=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.998, accuracy_score=0.98805


In [43]:
rfc_probs

array([[0.79269889, 0.20730111],
       [0.62647589, 0.37352411],
       [0.39858304, 0.60141696],
       ...,
       [0.85499379, 0.14500621],
       [0.64097246, 0.35902754],
       [0.9903395 , 0.0096605 ]])

In [45]:
probs

array([0.79269889, 0.62647589, 0.39858304, ..., 0.85499379, 0.64097246,
       0.9903395 ])

In [34]:
from sklearn.metrics import roc_curve, auc
from numpy import argmax

preds = rfc_probs[:,1]
fpr, tpr, threshold = roc_curve(yTest, preds)
roc_auc = auc(fpr, tpr)

# get the best threshold
J = tpr - fpr
ix = argmax(J)
best_thresh = threshold[ix]
print('Best Threshold=%f' % (best_thresh))

Best Threshold=0.550770


In [ ]:
df["DT_OP_GMT"] = pd.to_datetime(df["DT_OP_GMT"])

start_datetr = '2016-03-01'
end_datetr = '2016-03-15'

mask_tr = (df['DT_OP_GMT'] >= start_datetr) & (df['DT_OP_GMT'] < end_datetr) 
df_train = df.loc[mask_tr]

print(df_train)

In [ ]:
start_datevl = '2016-03-14'
end_datevl = '2016-03-22'

#greater than the start date and smaller than the end date
mask = (df['DT_OP_GMT'] >= start_datets) & (df['DT_OP_GMT'] < end_datets)
df_test = df.loc[mask]

In [ ]:
xTrain = df_train[['TIME_ORA_OP','ID_TIPO_CARTA','DEV_MEDIA_IMP_TC_1M', 'PERC_RISK_CIRC_TC_TO_1M',
       'DEV_MEDIA_IMP_CAN_1M', 'DEV_MEDIA_IMP_TO_1M', 'PERC_OPER_NON_AUTOR_1M',
       'PERC_OPER_NON_AUTOR_2M', 'MEDIA_NON_AUT_SEC_4',
       'DEV_IMP_MEDIA_ULTIME_3', 'PERC_RISK_MCC_1M', 'FLAG_IMP_PREC_0',
       'TREND_3_5','FLAG_CAMBIO_PAESE_NON_AUT',
       'PERC_ESITO_NSF_G', 'PERC_ESITO_CV2_G', 'PERC_ESITO_PIN_G',
       'PERC_ESITO_SCUDO_G', 'PERC_ESITO_DT_SCAD_G', 'PERC_ESITO_ALTRO_G',
       'DEV_MEDIA_AUT_SEC_4','IMP_TRZ_RECON']]

xTest = df_test[['TIME_ORA_OP','ID_TIPO_CARTA','DEV_MEDIA_IMP_TC_1M', 'PERC_RISK_CIRC_TC_TO_1M',
       'DEV_MEDIA_IMP_CAN_1M', 'DEV_MEDIA_IMP_TO_1M', 'PERC_OPER_NON_AUTOR_1M',
       'PERC_OPER_NON_AUTOR_2M', 'MEDIA_NON_AUT_SEC_4',
       'DEV_IMP_MEDIA_ULTIME_3', 'PERC_RISK_MCC_1M', 'FLAG_IMP_PREC_0',
       'TREND_3_5','FLAG_CAMBIO_PAESE_NON_AUT',
       'PERC_ESITO_NSF_G', 'PERC_ESITO_CV2_G', 'PERC_ESITO_PIN_G',
       'PERC_ESITO_SCUDO_G', 'PERC_ESITO_DT_SCAD_G', 'PERC_ESITO_ALTRO_G',
       'DEV_MEDIA_AUT_SEC_4','IMP_TRZ_RECON']]

yTrain = df_train["FLAG_FRODE"]

yTest = df_test["FLAG_FRODE"]

In [ ]:
#building the Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier 
#random forest model creation 
rfc = RandomForestClassifier(bootstrap = True, random_state=0,
                             max_features = 2, max_depth = 14, n_estimators = 200, class_weight = 'balanced')
rfc.fit(xTrain,yTrain)
predictions = rfc.predict(xTest)
print(accuracy_score(yTest, predictions))
print(confusion_matrix(yTest, predictions))
print(classification_report(yTest, predictions))

In [ ]:
#building the Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier 
#random forest model creation 
rfc = RandomForestClassifier(bootstrap = True, random_state=0,
                             max_features = 2, max_depth = 14, n_estimators = 200, class_weight = 'balanced_subsample')
rfc.fit(xTrain,yTrain)
predictions = rfc.predict(xTest)
print(accuracy_score(yTest, predictions))
print(confusion_matrix(yTest, predictions))
print(classification_report(yTest, predictions))

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [6,8,10],
    'n_estimators': [100, 200, 300]
}
# Create a based model
xgb = XGBClassifier(random_state=0,n_jobs=-1,objective='binary:logistic', eval_metric = 'logloss',use_label_encoder =False)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgb, param_grid = param_grid, 
                          cv = 3)

In [ ]:
# Fit the grid search to the data
grid_search.fit(xTrain, yTrain)
grid_search.best_params_

In [ ]:
print (f'Train Accuracy - : {grid_search.score(xTrain,yTrain):.3f}')
print (f'Test Accuracy - : {grid_search.score(xValid,yValid):.3f}')

In [ ]:
#For XGB
# Make predictions on validation dataset
XGB = XGBClassifier(random_state=0,n_jobs=-1,use_label_encoder =False, eval_metric = 'logloss')
XGB.fit(xTrain,yTrain)
predictions_xgb = XGB.predict(xTest)
print(accuracy_score(yTest, predictions_xgb))
print(confusion_matrix(yTest, predictions_xgb))
print(classification_report(yTest, predictions_xgb))

In [ ]:
#For XGB
# Make predictions on validation dataset
XGB = XGBClassifier(random_state=0,n_jobs=-1,objective='binary:logistic', 
                    max_depth = 6, n_estimators = 100, eval_metric = 'logloss',use_label_encoder =False)
XGB.fit(xTrain,yTrain)
predictions_xgb = XGB.predict(xTest)
print(accuracy_score(yTest, predictions_xgb))
print(confusion_matrix(yTest, predictions_xgb))
print(classification_report(yTest, predictions_xgb))

In [ ]:
xgb.get_params().keys()

In [ ]:
rf.get_params().keys()